# Cert-Elastic Colab ワークフロー

A100 GPU での評価を想定したノートブックです。上から順番に実行すれば、環境構築から最終評価までを一度で行えます。

## ノートブック構成
- 1. 初期設定 / Setup
- 2. 超軽量ロジック確認テスト (Smoke Tests)
- 3. 本番実行 (Full Evaluation)
- Appendix: 追加のデバッグユーティリティ

## 1. 初期設定 / Setup
Colab セッションを立ち上げた直後は、以下のセルを順番に実行してください。

### 1.1 ハードウェアとバージョン確認
GPU が正しく割り当てられているか、主要ライブラリのバージョンを確認します。

In [ ]:
!python -V
!pip -V
!nvidia-smi || echo "No NVIDIA GPU"

import platform, torch
print("torch:", torch.__version__, "cuda available:", torch.cuda.is_available())
print("python:", platform.python_version())

### 1.2 Google Drive をマウントして環境を定義
実行ログやキャッシュを Drive に保存して、次回以降の Colab 起動を高速化します。

In [ ]:
from google.colab import drive
import os, pathlib

drive.mount('/content/drive')
DRIVE_ROOT = '/content/drive/MyDrive'
PROJECT_NAME = 'Cert-Elastic'
PROJECT_DIR = f'{DRIVE_ROOT}/{PROJECT_NAME}'
print('PROJECT_DIR =', PROJECT_DIR)

# Hugging Face キャッシュを Drive に置いて再利用する
os.environ.setdefault('HF_HOME', f'{DRIVE_ROOT}/.cache/huggingface')
os.environ.setdefault('HF_DATASETS_CACHE', f"{os.environ['HF_HOME']}/datasets")
os.environ.setdefault('TRANSFORMERS_CACHE', f"{os.environ['HF_HOME']}/hub")
for key in ('HF_HOME', 'HF_DATASETS_CACHE', 'TRANSFORMERS_CACHE'):
    pathlib.Path(os.environ[key]).mkdir(parents=True, exist_ok=True)
print('HF caches under:', os.environ['HF_HOME'])

### 1.3 ワークスペースを用意
Drive 上に成果物用ディレクトリを作成し、必要であればローカルからコードを同期します。

- まだリポジトリを配置していない場合は `!git clone` や `!rsync` などで Drive の `Cert-Elastic/` にコピーしてください。
- 以下のセルは成果物用のサブディレクトリを初期化します。

In [ ]:
import pathlib

required = ['runs', 'outputs', 'logs', 'models', 'data']
for rel in required:
    path = pathlib.Path(PROJECT_DIR) / rel
    path.mkdir(parents=True, exist_ok=True)
print('Ensured directories:', required)

### 1.4 依存ライブラリのインストール
Colab に必要な Python パッケージをインストールします。GPU Wheel は CUDA 12.4 用を利用します。

In [ ]:
%cd "$PROJECT_DIR"
!python -m pip install -U pip wheel setuptools
!pip install -U "jedi>=0.16"
!pip install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio
!pip install -r requirements.txt

import datasets
try:
    import lm_eval
    lm_eval_version = getattr(lm_eval, '__version__', 'unknown')
except ImportError:
    lm_eval_version = 'not installed'
print('datasets:', datasets.__version__)
print('lm-eval:', lm_eval_version)

### 1.5 Python パスとモジュール読み込み確認
`cert_elastic` が正しく import できることを確認します。

In [ ]:
import os, sys

if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)
print('sys.path[0:3] =', sys.path[:3])

try:
    import cert_elastic
    print('cert_elastic imported from:', cert_elastic.__file__)
except Exception as exc:
    print('import error:', exc)

### 1.6 Hugging Face Hub へのログイン
Read 専用トークンを使用して Hugging Face にログインします。トークンは環境変数 `HF_TOKEN` に保持され、キャッシュに保存されます。

In [ ]:
import os
from getpass import getpass

try:
    from huggingface_hub import login, whoami
except ImportError as exc:
    raise RuntimeError('huggingface_hub が見つかりません。requirements のインストールを確認してください。') from exc

if os.environ.get('HF_TOKEN'):
    print('HF_TOKEN is already set; skipping interactive login.')
else:
    token = getpass('Enter your Hugging Face token (read access): ').strip()
    if token:
        login(token=token, add_to_git_credential=True)
        os.environ['HF_TOKEN'] = token
        try:
            info = whoami()
            print('Logged in as:', info.get('name') or info.get('preferred_username'))
        except Exception:
            print('Login succeeded.')
    else:
        print('No token provided. Some datasets may require authentication.')

### 1.7 データセットのウォームアップ (任意)
Hugging Face Datasets ライブラリから直接ダウンロードし、キャッシュしておきます。既にキャッシュ済みの場合は短時間で終了します。

In [ ]:
import os
from datasets import load_dataset

cache_kwargs = {"trust_remote_code": True}
token = os.environ.get('HF_TOKEN')
if token:
    cache_kwargs['token'] = token
cache_dir = os.environ.get('HF_DATASETS_CACHE') or os.environ.get('HF_HOME')
if cache_dir:
    cache_kwargs['cache_dir'] = cache_dir
print('cache kwargs:', cache_kwargs)

def prefetch(label, fn, allow_fail=False):
    try:
        fn()
        print('[prefetch] ok:', label)
    except Exception as err:  # noqa: BLE001
        msg = f"[prefetch] warn: {label} failed -> {err}"
        if allow_fail:
            print(msg)
        else:
            raise RuntimeError(msg) from err

prefetch('openai/gsm8k:main', lambda: load_dataset('openai/gsm8k', 'main', **cache_kwargs))
for cfg in ['algebra','counting_and_probability','geometry','intermediate_algebra','number_theory','prealgebra','precalculus']:
    prefetch(f'EleutherAI/hendrycks_math:{cfg}', lambda cfg=cfg: load_dataset('EleutherAI/hendrycks_math', cfg, split='train', **cache_kwargs), allow_fail=True)
prefetch('google-research-datasets/mbpp:sanitized', lambda: load_dataset('google-research-datasets/mbpp', 'sanitized', **cache_kwargs))
prefetch('openai/openai_humaneval', lambda: load_dataset('openai/openai_humaneval', **cache_kwargs), allow_fail=True)

print('Datasets cached under:', cache_dir or '~/.cache/huggingface/datasets')

## 2. 超軽量ロジック確認テスト (Smoke Tests)
本番前に配線と依存関係をチェックするための最小構成テストです。

### 2.1 `run_evaluate_cert.py` の高速テスト
bf16 / SDPA 構成で最小本数のプロンプトを評価し、ログが生成されることを確認します。

In [ ]:
%cd "$PROJECT_DIR"

MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.3'
DTYPE = 'bfloat16'
LOAD_IN_4BIT = 0
ATNN_IMPL = 'sdpa'
MAX_NEW_TOKENS = 128
EVAL_PROMPTS_N = 32
RUN_FAST = 1

!python run_evaluate_cert.py \
  --model_id {MODEL_ID} \
  --dtype {DTYPE} \
  --load_in_4bit {LOAD_IN_4BIT} \
  --device_map auto \
  --attn_impl {ATNN_IMPL} \
  --max_new_tokens {MAX_NEW_TOKENS} \
  --eval_prompts_n {EVAL_PROMPTS_N} \
  --run_fast {RUN_FAST} \
  --out_dir runs


### 2.2 フルパイプラインのスモークテスト
Colab 用ドライバースクリプトを `--smoke-test` 付きで実行し、全工程が動作するかを確認します。

In [ ]:
%cd "$PROJECT_DIR"
!python colab/run_full_eval.py \
  --model-id mistralai/Mistral-7B-Instruct-v0.3 \
  --device-map cuda:0 \
  --attn-impl sdpa \
  --smoke-test \
  --math-probe


## 3. 本番実行 (Full Evaluation)
ここからが A100 前提の本番パラメータです。必要に応じてモデル ID や生成長を調整してください。

### 3.1 すべての評価タスクを実行
GSM8K / Hendrycks MATH / MBPP / HumanEval を Hugging Face Datasets から直接読み込み、完全な評価を行います。

In [ ]:
%cd "$PROJECT_DIR"

MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.3'
DTYPE = 'bfloat16'
ATNN_IMPL = 'sdpa'
EPSILON = 0.02
ALPHA = 0.5
BETA = 2.0
TASKS = 'gsm8k,humaneval,mbpp,hendrycks_math'
MAX_NEW_TOKENS = 256
EVAL_PROMPTS = 64
PAPER_N_ITEMS = 256
PAPER_GEN_LEN = 256

!python colab/run_full_eval.py \
  --model-id {MODEL_ID} \
  --dtype {DTYPE} \
  --device-map auto \
  --attn-impl {ATNN_IMPL} \
  --epsilon {EPSILON} \
  --alpha {ALPHA} \
  --beta {BETA} \
  --max-new-tokens {MAX_NEW_TOKENS} \
  --eval-prompts {EVAL_PROMPTS} \
  --tasks {TASKS} \
  --paper-n-items {PAPER_N_ITEMS} \
  --paper-gen-len {PAPER_GEN_LEN} \
  --out_dir runs


### 3.2 直近の出力を要約
`runs/run_*` ディレクトリを確認し、主要な JSON / CSV を冒頭だけ表示します。

In [ ]:
%cd "$PROJECT_DIR"
import glob, json
from pathlib import Path

run_dirs = sorted(glob.glob('runs/run_*'))
print('recent runs:', run_dirs[-3:])
if run_dirs:
    latest = Path(run_dirs[-1])
    print('latest run:', latest)
    for name in ['config.eval.json','summary.json','results.logging.json','cert_checks.csv','lmeval_baseline.json','lmeval_cert.json']:
        path = latest / name
        if not path.exists():
            print(f'{path} not found')
            continue
        print(f'--- {path} (head) ---')
        if path.suffix == '.json':
            try:
                data = json.loads(path.read_text() or '{}')
                preview = json.dumps(data, ensure_ascii=False)[:800]
            except Exception:
                preview = path.read_text()[:800]
            print(preview)
        else:
            print(''.join(path.read_text().splitlines()[:10]))
else:
    print('No runs yet.')

## Appendix: 追加ユーティリティ
必要に応じてロングランのログをファイルに保存したい場合に利用してください。

In [ ]:
%cd "$PROJECT_DIR"
import os, subprocess, sys
from pathlib import Path

log_path = Path('runs/lmeval_debug.log')
os.makedirs(log_path.parent, exist_ok=True)

cmd = [
    sys.executable, '-m', 'bench.run_lmeval_cert',
    '--model_id', 'mistralai/Mistral-7B-Instruct-v0.3',
    '--dtype', 'bfloat16',
    '--attn_impl', 'sdpa',
    '--epsilon', '0.02',
    '--alpha', '0.5',
    '--beta', '2.0',
    '--tasks', 'gsm8k,humaneval,mbpp,hendrycks_math',
    '--fewshot', '0',
    '--limit', '0',
    '--out_dir', 'runs'
]

print('Running:', ' '.join(cmd))
print('Logging to:', log_path)

with open(log_path, 'wb') as fh:
    proc = subprocess.Popen(cmd, stdout=fh, stderr=subprocess.STDOUT)
    try:
        proc.wait()
    except KeyboardInterrupt:
        proc.terminate()
        raise

print('Return code:', proc.returncode)
if log_path.exists():
    tail = log_path.read_text(errors='replace')[-20000:]
    print('--- log tail ---')
    print(tail)
else:
    print('log file missing')